In [ ]:
# 1. Imports
import os
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
RAW_DIR = "../data/brats20"  # adjust after extraction

In [ ]:
# 3. Utility to visualize an MRI slice
def show_slice(volume, slice_idx):
    plt.imshow(volume[:, :, slice_idx], cmap='gray')
    plt.axis('off')